### Horizon Line Detection

This notebook carries out horizon line detection using the Canny Edge Detection that comes with OpenCV.

Sources:
- [1] https://www.pyimagesearch.com/2015/04/06/zero-parameter-automatic-canny-edge-detection-with-python-and-opencv/

In [26]:
import sys
from pathlib import Path
import cv2
import numpy as np
import math

# Define paths to sub-folders
root_dir = Path.cwd()
images_path = root_dir / '..' / 'test_images'

In [50]:
def canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)
    # apply automatic Canny edge detection using the computed median
    lower = 225 #int(max(0, (1.0 - sigma) * v))
    upper = 250 #int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)
    # return the edged image
    return edged

def plot_line(img, rho, theta):
    # Plots the line coming out of a Hough Line Transform
    a = math.cos(theta)
    b = math.sin(theta)
    x0 = a * rho
    y0 = b * rho
    pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
    pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
    cv2.line(img, pt1, pt2, (255,0,0), 5, cv2.LINE_AA)

In [51]:
for image_path in list(images_path.glob('*.png')):
    image = cv2.imread(str(image_path))
    # Blur image and convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (6, 6), 0)
    # Use Canny edge detection to find edges
    edges = auto_canny(blurred)
    # Use Hough Line Transform to get lines
    threshold = 1200
    lines = cv2.HoughLines(edges, 30, np.pi / 90, threshold, None, 0, 0)
    if lines is not None:
        print('Found %s lines' % (len(lines)))
#         # Print all lines
#         for line in lines:
#             for rho,theta in line:
#                 plot_line(edges, rho, theta)
        # Average line
        avg_rho = np.mean([line[0][0] for line in lines])
        avg_theta = np.mean([line[0][1] for line in lines])
        plot_line(edges, avg_rho, avg_theta)
    else:
        print('No Horizon Found')

#     cv2.imshow("Original", image)
    cv2.imshow("Edges", edges)
    cv2.waitKey(0)

No Horizon Found
Found 441 lines
Found 110 lines
No Horizon Found
Found 367 lines
No Horizon Found
Found 318 lines
Found 332 lines
Found 227 lines
Found 38 lines
No Horizon Found
Found 1 lines
Found 284 lines
Found 274 lines
Found 134 lines
Found 103 lines
Found 17 lines
Found 1 lines
Found 34 lines
Found 314 lines
Found 376 lines
Found 381 lines
Found 30 lines
Found 1 lines
Found 42 lines
Found 316 lines
Found 75 lines
Found 32 lines
Found 118 lines
Found 294 lines
Found 115 lines
